In [44]:
from simulation.GraphSim import GraphSim
from utils.activation_functions import sigmoid
from copy import copy
import networkx as nx
import numpy as np
from scipy.stats import spearmanr

In [16]:
def sim_graph_sequence(graph_name: str, n_sim: int, n_nodes: int, rho: float, seed: int=None):

    gs = GraphSim(graph_name=graph_name)

    # gen seed
    gs.update_seed(seed=seed)

    ps_orig_out = []
    ps_out = []
    rhos_out = []
    graphs_out = []
    for i in range(n_sim):

        # generate probability of edge creation
        ps = gs.get_p_from_bivariate_gaussian(s=rho, size=1)
        ps_orig = copy(ps)
        ps = sigmoid(ps)
        j = 0

        if graph_name == 'erdos_renyi':
            graph1 = gs.simulate_erdos(n=n_nodes, prob=ps[j,0])
            graph2 = gs.simulate_erdos(n=n_nodes, prob=ps[j,1])
        elif graph_name == 'k_regular':
            graph1 = gs.simulate_k_regular(n=n_nodes, k=int(10*ps[j,0]))
            graph2 = gs.simulate_k_regular(n=n_nodes, k=int(10*ps[j,1]))
        elif graph_name == 'geometric':
            graph1 = gs.simulate_geometric(n=n_nodes, radius=ps[j,0])
            graph2 = gs.simulate_geometric(n=n_nodes, radius=ps[j,1])
        elif graph_name == 'barabasi_albert':
            graph1 = gs.simulate_barabasi_albert(n=n_nodes, m=int(10*ps[j,0]))
            graph2 = gs.simulate_barabasi_albert(n=n_nodes, m=int(10*ps[j,1]))
        elif graph_name == 'watts_strogatz':
            graph1 = gs.simulate_watts_strogatz(n=n_nodes, k=3, p=ps[j,0])
            graph2 = gs.simulate_watts_strogatz(n=n_nodes, k=3, p=ps[j,1])
        else:
            raise Exception("Graph not present")

        graphs_out.append([graph1, graph2])
        ps_out.append(ps)
        ps_orig_out.append(ps_orig)
        rhos_out.append(rho)
    
    return graphs_out, ps_out, ps_orig_out, rhos_out

In [49]:
rho = 0.5

graphs, ps, ps_orig, rhos =  sim_graph_sequence(graph_name="erdos_renyi",
                                                n_sim=10,
                                                n_nodes=10,
                                                rho=rho,
                                                seed=19940202)

# 1) Benchmark Methods


## Spectrum

In [57]:
max_eigenvalues = []
for sample in graphs:

    # subset graphs
    graph1 = sample[0]
    graph2 = sample[1]

    # get adjacency info
    adj1 = nx.adjacency_matrix(graph1).toarray()
    adj2 = nx.adjacency_matrix(graph2).toarray()

    # compute max eigenvalues
    max_eigen1 = np.max(np.linalg.eigvalsh(adj1))
    max_eigen2 = np.max(np.linalg.eigvalsh(adj2))

    max_eigenvalues.append([max_eigen1, max_eigen2])
max_eigenvalues = np.array(max_eigenvalues)

spectrum_correl = np.round(spearmanr(max_eigenvalues[:,0], max_eigenvalues[:,1]).correlation, 3)
squared_dis = np.round((rho - spectrum_correl) ** 2, 3)

print(f'Spectrum correlation is {spectrum_correl}, True correlation is {rho}, Squared distance is {squared_dis}')

Spectrum correlation is 0.638, True correlation is 0.5, Squared distance is 0.019


## Frobenius distance

In [78]:
fro_norm_all = []
fro_correl_all = []
for sample in graphs:

    # subset graphs
    graph1 = sample[0]
    graph2 = sample[1]

    # get adjacency info
    adj1 = nx.adjacency_matrix(graph1).toarray()
    adj2 = nx.adjacency_matrix(graph2).toarray()

    adj_dis = adj1 - adj2
    fro_norm = np.linalg.norm(adj_dis, ord="fro")
    fro_correl = sigmoid(fro_norm)

    fro_norm_all.append(fro_norm)
    fro_correl_all.append(fro_correl)
fro_norm_all = np.array(fro_norm_all)
fro_correl_all = np.array(fro_correl_all)
fro_correl = np.mean(fro_correl_all)
squared_dis = np.round((rho - fro_correl) ** 2, 3)

print(f'Frobenius correlation is {fro_correl}, True correlation is {rho}, Squared distance is {squared_dis}')

Frobenius correlation is 0.8845602473215528, True correlation is 0.5, Squared distance is 0.148
